In [3]:
import math
import numpy as np
import numba
from numba import jit, cuda, vectorize, float64

# 1 First testing numba

In [3]:
cov = np.random.randn(100).reshape((10, 10))
vec = np.random.randn(10)

In [4]:
np.dot(np.dot(vec, cov), vec)

-9.2643887216958039

In [5]:
@numba.jit(nopython=True)
def sum2d(M):
    summation = 0.
    for i in range(M):
        for j in range(M):
            summation += np.dot(np.dot(vec, cov), vec)
    return summation
%timeit sum2d(5000)

5.44 s ± 52.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def sum2d(M):
    summation = 0.
    for i in range(M):
        for j in range(M):
            summation += np.dot(np.dot(vec, cov), vec)
    return summation
%timeit sum2d(5000)

31.1 s ± 998 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 2 Need NVIDIA gpu to run CUDA

## Below runs on i7-7700K and NVIDIA TITAN Xp, a powerful GPU over a thousand dollars

In [3]:
print(cuda.gpus)

<Managed Device 0>


## 2.1 Vectorize_add
### It seems that target=cpu is better than target=cuda. Unexpected.

In [3]:
@vectorize([float64(float64,float64)], target='cuda')
def add_vec_cuda(a, b):
    return a + b

In [4]:
@vectorize([float64(float64, float64)], target='cpu')
def add_vec_cpu(a, b):
    return a + b

In [5]:
@jit(nopython=True)
def add_jit(X, Y):
    size = len(X)
    Z = []
    for i in range(size):
        Z.append(X[i] + Y[i])
    return Z

In [6]:
a = np.arange(100000, dtype=np.float64)
%timeit add_vec_cuda(a, a)
%timeit add_vec_cpu(a, a)
%timeit add_jit(a, a)

1.1 ms ± 272 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
27.2 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.32 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
a = np.arange(1000000, dtype=np.float64)
%timeit add_vec_cuda(a, a)
%timeit add_vec_cpu(a, a)
%timeit add_jit(a, a)

4.62 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
644 µs ± 9.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
28.4 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
a = np.arange(10000000, dtype=np.float64)
%timeit add_vec_cuda(a, a)
%timeit add_vec_cpu(a, a)
%timeit add_jit(a, a)

59 ms ± 985 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.4 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
344 ms ± 658 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
a = np.arange(100000000, dtype=np.float64)
%timeit add_vec_cuda(a, a)
%timeit add_vec_cpu(a, a)
%timeit add_jit(a, a)

549 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
256 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.39 s ± 9.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2.2 Vectorize_sincos

In [ ]:
n = 1000000
x = np.arange(n, dtype=np.float64)
y = np.arange(n, dtype=np.float64)

np_ans = np.sin(x) * np.cos(y)
nb_cpu_ans = cpu_sincos(x, y)
nb_gpu_ans = gpu_sincos(x, y)

print("CPU vectorize:", np.allclose(nb_cpu_ans, np_ans))
print("CPU vectorize:", np.allclose(nb_gpu_ans, np_ans))

array([ 0.        ,  0.45464871, -0.37840125, ..., -0.20931808,
        0.49999812, -0.2068272 ])

array([ 0.        ,  0.45464871, -0.37840125, ..., -0.20931808,
        0.49999812, -0.2068272 ])

## 2.1 Kernal

In [7]:
@cuda.jit
def my_kernal(io_array):
    pos = cuda.grid(1)
    if pos < io_array.size: # check array boundaries
        io_array[pos] *= 2  # do the computation
        
data = np.ones(500000000)
threadsperblock = 256
blockspergrid = math.ceil(data.shape[0]/threadsperblock)
print(blockspergrid)
my_kernal[blockspergrid, threadsperblock](data)
data

1953125


array([2., 2., 2., ..., 2., 2., 2.])

## 2.2 Matrix Multiplication

In [30]:
@cuda.jit
def matmul(A, B, C):
    row, col = cuda.grid(2)
    #print(row, col)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [35]:
def host_code():
    A = np.full((24, 12), 3, np.float)
    B = np.full((12, 6), 4, np.float)
    #copy the arrays to the device
    A_global_mem = cuda.to_device(A)
    B_global_mem = cuda.to_device(B)
    
    #allocate memory on the device for the result
    C_global_mem = cuda.device_array((24, 6))
    
    # configure the blocks
    threadsperblock = (16, 16)
    blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    print(blockspergrid)
    
    #start the kernal
    matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
    
    C = C_global_mem.copy_to_host()
    print(C)

In [36]:
host_code()

(2, 1)
[[144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]
 [144. 144. 144. 144. 144. 144.]]


In [2]:
# the computation will be done on blocks of TPB*TPB elements
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    '''matrix multiplication C = A*B
       each thread computes one element of the result matrix C
    '''
    # define an array in the shared memory
    # the size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    
    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] or y >= C.shape[1]:
        return
    
    # each thread computes one element in the result matrix
    # the dot product is chunked into dot products of TPB-long vectors
    tmp = 0
    for i in range(int(A.shape[1]/TPB)):
        # preload data into shared memory
        sA[tx, ty] = A[x, ty + i*TPB]
        sB[tx, ty] = B[tx + i*TPB, y]
        
        # wait until all threads finish preloading
        cuda.syncthreads()
        
        # computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j]*sB[j, ty]
        
        # wait until all threads finish computing
        cuda.syncthreads()
    
    C[x, y] = tmp
    
    
def host_code():
    # initialize the data arrays
    A = np.full((TPB*2, TPB*3), 3, np.float64)  # [32 x 48] matrix containing all 3's
    B = np.full((TPB*3, TPB*1), 4, np.float64)  # [48 x 16] matrix containing all 4's
    
    A_global_mem = cuda.to_device(A)
    B_global_mem = cuda.to_device(B)
    C_global_mem = cuda.device_array((TPB*2, TPB*1))  # [32 x 16] matrix result
    
    # configure the blocks
    threadsperblock = (TPB, TPB)
    blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
    blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
    blockspergrid   = (blockspergrid_x, blockspergrid_y)
    
    # start the kernal
    fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
    
    res = C_global_mem.copy_to_host()
    print(res)

In [3]:
host_code()

[[576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576.
  576. 576.]
 [576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576. 576

In [2]:
my_gpu = numba.cuda.get_current_device()

In [10]:
print('Running on GPU:', my_gpu.name)
cc = my_gpu.compute_capability
print('Compute capability:', '%d.%d' % cc)
majorcc = cc[0]
print('Number of streaming multiprocessor:', my_gpu.MULTIPROCESSOR_COUNT)
print('Warp size:', my_gpu.WARP_SIZE)

Running on GPU: b'TITAN Xp'
Compute capability: 6.1
Number of streaming multiprocessor: 30
Warp size: 32


# Test Reduction

In [4]:
my_gpu = numba.cuda.get_current_device()
print('Running on GPU:', my_gpu.name)

Running on GPU: b'GeForce GTX 1070'


In [93]:
@cuda.reduce
def sum_reduce(a, b):
    return a+b

In [100]:
A = np.random.rand(1000) * 1e-200
A

array([8.99243252e-201, 8.12433843e-202, 2.06057770e-201, 2.11568071e-201,
       1.55664446e-201, 9.66401485e-201, 9.20515738e-201, 7.72448510e-203,
       1.87608744e-201, 2.20562155e-201, 2.35507333e-201, 9.38715782e-201,
       1.62888949e-201, 7.07204916e-201, 8.22117692e-201, 6.87778031e-202,
       1.99544743e-201, 1.48369256e-202, 7.69158783e-201, 5.78088619e-201,
       6.05913133e-201, 2.70405787e-202, 6.44419141e-201, 5.35446603e-201,
       7.14491557e-201, 5.63080554e-201, 4.03097434e-201, 2.63886607e-201,
       7.14761070e-201, 9.55237499e-201, 5.74633931e-202, 1.29778028e-201,
       7.45739347e-201, 1.07503370e-201, 7.48496233e-201, 2.61831226e-201,
       6.42456894e-201, 5.61207085e-201, 1.51400501e-201, 4.74511242e-201,
       2.04818946e-201, 2.78280515e-201, 7.67783910e-201, 6.14282350e-201,
       7.86927547e-201, 1.16825348e-201, 9.27018507e-201, 6.33715915e-201,
       4.19510649e-201, 9.30240243e-201, 5.99176830e-201, 9.59921284e-201,
       6.17749862e-202, 9

In [101]:
A.sum()

5.047549521337438e-198

In [102]:
d_A = cuda.to_device(A)

In [103]:
sum_reduce(d_A)

5.047549521337438e-198

In [81]:
%timeit A.sum()

11.2 µs ± 44.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [37]:
%timeit sum_reduce(A)

11.4 ms ± 860 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
%timeit sum_reduce(d_A)

1.67 ms ± 6.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


9.999999999999997e-17

In [46]:
sum_reduce(d_A)

TypeError: only support 1D array

In [39]:
sum_reduce(A)

3276801280000.0